#prediction the number of mosqiutos
## preparing the data

In [242]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics

In [243]:
df_train = pd.read_csv('aggregated_train_with_nummosquitos.csv')# from make_aggregated_with_Nummosquitos
#df_test = pd.read_csv('../test_filled_new.csv')
#orig = pd.read_csv('../test_filled_new.csv')
df_test = pd.read_csv('../test_filled_new.csv')

### delete string vars

In [244]:
'Species' in df_test.columns

True

In [245]:
[var for var in df_train.columns if var not in df_test.columns]

['Unnamed: 0', 'NumMosquitos', 'WnvPresent']

In [246]:
X = df_train.copy()
#X.dtypes

In [247]:
[var for var in df_test.columns if var not in X.columns]

['Id']

In [248]:
df_test.Species.values

array(['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS', ...,
       'CULEX TARSALIS', 'UNSPECIFIED CULEX', 'CULEX ERRATICUS'], dtype=object)

In [249]:
string_features = ['Date','Address','Street','AddressNumberAndStreet','CodeSum']
species_encoder = preprocessing.LabelEncoder()
trap_encoder = preprocessing.LabelEncoder()
#X_dump = X[string_features]
df_dump = df_test[string_features]
#X_dump = X[string_features]

df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
X.Species = species_encoder.fit_transform(X.Species)
y = df_train['NumMosquitos']
df_test.Species = species_encoder.transform(df_test.Species)
df_train.Species = species_encoder.transform(df_train.Species)


/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [250]:

df_test.Trap = trap_encoder.fit_transform(df_test.Trap)
X.Trap = trap_encoder.transform(X.Trap)
df_train.Trap = trap_encoder.transform(df_train.Trap)

try:
 X = X.drop(string_features,axis=1)
except:
    pass
try:
 df_test = df_test.drop(string_features,axis=1)
except:
    pass
try:
 df_test = df_test.drop('Id',axis=1)
except: 
    pass
#X.head()
try:
    X = X.drop(['Unnamed: 0', 'NumMosquitos', 'WnvPresent'],axis=1)
except:
        pass

In [251]:
df_dump.head()

,Date,Address,Street,AddressNumberAndStreet,CodeSum
0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",


In [252]:
X.dtypes[:70]

Species              int64
Block                int64
Trap                 int64
Latitude           float64
Longitude          float64
AddressAccuracy      int64
Station              int64
Tmax                 int64
Tmin                 int64
Tavg               float64
Depart             float64
DewPoint             int64
WetBulb            float64
Heat                 int64
Cool                 int64
...
13_week_avrgPrecipTotal    float64
13_week_avrgTavg           float64
14_week_avrgPrecipTotal    float64
14_week_avrgTavg           float64
15_week_avrgPrecipTotal    float64
15_week_avrgTavg           float64
16_week_avrgPrecipTotal    float64
16_week_avrgTavg           float64
17_week_avrgPrecipTotal    float64
17_week_avrgTavg           float64
18_week_avrgPrecipTotal    float64
18_week_avrgTavg           float64
19_week_avrgPrecipTotal    float64
19_week_avrgTavg           float64
20_week_avrgPrecipTotal    float64
Length: 70, dtype: object

In [253]:
[var for var in X.columns if var not in df_test.columns]

[]

In [254]:
[var for var in df_test.columns if var not in X.columns]

[]

##Cross validation over several regressors
Already tried: SGD, Ridge, Kneighbours. They all return NEGATIVE $R^2$, indicating that they are worse than just using the mean value. 

Finding some better features might be necessary.

In [289]:
from sklearn.cross_validation import train_test_split, ShuffleSplit,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, SGDRegressor
import numpy as np

###using rf,ridge

In [290]:

clf_list= [#RandomForestRegressor(n_estimators=100,n_jobs=-1),
           #RandomForestRegressor(n_estimators=400,n_jobs=-1),
           #RandomForestRegressor(n_estimators=1000,n_jobs=-1), 
           #Ridge(alpha = .01,normalize=True),
           #Ridge(alpha = .1,normalize=True),
           #Ridge(alpha = .1),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=7),
           GradientBoostingRegressor(n_estimators = 100,max_depth=10),
           GradientBoostingRegressor(n_estimators = 100,max_depth=15,subsample = .7),#so far the best one, n_es = 400
           GradientBoostingRegressor(n_estimators = 100,max_depth=15),
           GradientBoostingRegressor(n_estimators = 100,max_depth=20,subsample = .7)
           #KNeighborsRegressor(),
           #SGDRegressor()
           #GradientBoostingRegressor(n_estimators = 100,max_depth=4),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=2)
          ]
random_state= 46

In [306]:
X = scale(X)

err = []
r2 = []
y_arr = pd.DataFrame(y_test)
i = 1
#clf = clf_list[1]
for clf in clf_list:
#for i in np.random.randint(1,size=4):
    i += 1
    X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)
    clf.fit(X_train,y_train)
    yhat = clf.predict(X_test)
    err.append( mean_squared_error(yhat,y_test))
    r2.append(r2_score(yhat,y_test))
    y_arr[i] = yhat
    


In [307]:
y_arr_mean = y_arr.drop(0,axis=1)
y_arr_mean = y_arr_mean.mean(axis=1)
y_arr_mean
r2_score(y_arr_mean,y_arr[0])

0.1068825460823245

In [308]:
err/ (y_test.mean()*len(yhat))

array([ 0.00362571,  0.00373775,  0.00427115,  0.00372012])

In [309]:
r2

[0.069445081044404278,
 0.10346879615148019,
 0.10468574694335442,
 0.12350359552310097]

In [310]:
np.array(err).mean(),np.array(err).std()

(171.07089861098677, 11.287887074739247)

In [311]:
np.array(r2).mean(),np.array(r2).std()

(0.10027580491558497, 0.019491707035611264)

In [312]:
err

[161.57992405201563, 166.57292859825191, 190.3436028749918, 165.78713891868773]

##use PCA before regression

In [291]:
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [300]:
#reduced_data = PCA(n_components = 50).fit_transform(X)
X = scale(X)
reduced_data = PCA(n_components = .99).fit_transform(X)


In [301]:
X.shape, reduced_data.shape

((10506, 77), (10506, 32))

In [302]:

err = []
r2 = []
y_arr = pd.DataFrame(y_test)
i = 1
clf = clf_list[1]
for clf in clf_list:
#for i in np.random.randint(1,size=4):
    i += 1
    X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)
    clf.fit(X_train,y_train)
    yhat = clf.predict(X_test)
    err.append( mean_squared_error(yhat,y_test))
    r2.append(r2_score(yhat,y_test))
    y_arr[i] = yhat
    



In [303]:
y_arr_mean = y_arr.drop(0,axis=1)
y_arr_mean = y_arr_mean.mean(axis=1)
y_arr_mean
r2_score(y_arr_mean,y_arr[0])

-1.4742055112611721

In [304]:
r2

[0.071637475518690596,
 0.11960055042794049,
 0.10631425288155616,
 0.12136154771431251]

In [305]:
err

[160.96186421180775,
 163.42393905761318,
 189.07645264315806,
 164.45545216335023]

#make the prediction


In [316]:
clf =            GradientBoostingRegressor(n_estimators = 100,max_depth=20,subsample = .7)
clf.fit(X,y)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=20, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2, n_estimators=100,
             random_state=None, subsample=0.7, verbose=0, warm_start=False)

In [317]:
y_test = clf.predict(scale(df_test))

#make file


In [320]:
df_test['NumMosquitos'] = y_test

df_test = pd.concat([df_test,df_dump,y],axis=1)

df_test.to_csv('aggregated_test_NumMosquito_no_strings.csv',index_label = 'Id')
#df_train = df_train.drop('Unnamed: 0',axis=1)
df_train.to_csv('aggregated_train_NumMosquito_no_strings.csv',index_label = 'Id')
